In [1]:
from lxml import etree as ET
import json
import os

In [8]:
#temp create template json without config 
test = open('30382939.xml')
jsontest = json.load(test)
#jsontest = json.load(open('30382939.xml'))

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
class XmlRecord():
    """XML-based record representation."""

    def modifyTag(self,item,ver):
        """ Modifies tag of an item if necessary."""
        tag = item.tag[self.config[ver]["NSL"]:]
        return tag

    def toNsString(self,ns, prefix=None):
        """Converts namespace to string prefix for element tags."""
        mif_ns = ns[prefix]
        mifstr = "{%s}" % mif_ns
        return mifstr

    def __init__(self, root=None, config=None):

        if root is not None:
            self.root = root
        else:
            self.root = {}

        self.config = {}
        if config is not None:

            for ver in config.keys():
                self.config[ver] = {}

                #"loads" in appropriate json file depending on format.
                #config is a deeply nested dictionary:
                #{version: {IN or OUT: {json file as a dictionary} } }
                #config[ver]["IN"] is a json file
                self.config[ver]["IN"] = json.load( open( config[ver]["IN"] ) )
                #what is NSL?
                self.config[ver]["NSL"] = len(self.config[ver]["IN"]["@NS"])+2

                self.config[ver]["OUT"] = json.load( open( config[ver]["OUT"] ) )

                # re-key  default ("*") namespace

                defns = None
                for nk in self.config[ver]["OUT"]["@NS"]:
                    if nk == "*":
                        defns = self.config[ver]["OUT"]["@NS"]["*"]
                if defns is not None:
                    self.config[ver]["OUT"]["@NS"].pop("*", None)
                    self.config[ver]["OUT"]["@NS"][None] = defns

    def parseXml(self, filename, ver, debug=False):

        template = self.config[ver]["IN"]

        #tree data structure from etree; holds the parsed data from passed in xml file.
        #returns elementTree object
        #documentobject structure
        recordTree = ET.parse( filename )
        #root object has a tag and attribute
        #root.tag, root.attrib
        #for child in root iterates over children of the root.
        #TODO: test this out with xml file to determine exactly how it works - what constitutes a "child" in the context of these xml files
        rootElem = recordTree.getroot()
        #template describes what to expect
        self.genericParse( template, ver, self.root, [] , rootElem, debug )
        return self
#elem is root element?
    def genericParse(self, template, ver, rec, rpath, elem, wrapped=False, debug=False):

        tag = self.modifyTag( elem, ver )

        #find corresponding template
        #else default in json
        #template is the dictionary representation of the appropriate json file, as pulled from the config dictionary
        #ttempl is now just the single row corresponding to appropriate tag
        if tag in template:
            ttempl = template[tag]
        else:
            ttempl = template["*"]

        if debug:
            print("\nTAG", tag, wrapped, len(rpath) )
            print(" TTEM", ttempl)

        #what is the relevence of asking if the root element has a parent; won't this always be false.
        #recursion^
        if elem.getparent() is not None:
            parentElem = elem.getparent()
            if wrapped:
                #going two levels up if current element is just a wrapper.
                if  parentElem.getparent() is not None:
                    parentElem = parentElem.getparent()
                else:
                    parentElem = None
        else:
            parentElem = None

        #just recording the parent tag
        if parentElem is not None:
            parentTag = self.modifyTag( parentElem, ver )
            if debug:
                print(" PAR", parentTag )

        else:
            parentTag = None
            if debug:
                print("PAR:  ROOT ELEM !!!")
        #print(parentTag)
        #noting down template for same attributes found under different parents;
        #this is the second level in the json files.
        if parentTag is not None and parentTag in ttempl:
            ctempl = ttempl[parentTag]
        else:
            ctempl = ttempl["*"]
        if debug:
            print("  CTEMPL", ctempl )

        # find wrap flag
        if "wrapper" in ctempl and ctempl["wrapper"] is not None:
            cwrap = ctempl["wrapper"]
        else:
            #default
            cwrap = template["*"]["*"]["wrapper"]
        #cwrap is a boolean, says whether current element is a wrapper
        if debug:
            print( "  CWRAP ", cwrap )

        if cwrap:
            #recursing one child deeper if current element is just a wrapper.
            for cchld in elem:
                if debug:
                    print("  CHLD",cchld.tag);

                self.genericParse( template, ver, rec, rpath, cchld, wrapped =True)
                if debug:
                    print( json.dumps(self.root, indent=2) )
            return

        # find current key:
        #checking the attributes, not the tags
        if "name" in ctempl and ctempl["name"] is not None:
            ckey = ctempl["name"]
        else:
            ckey = tag

        # find complex flag
        if "complex" in ctempl and ctempl["complex"] is not None:
            ccomplex = ctempl["complex"]
        else:
            #default
            ccomplex = template["*"]["*"]["complex"]

        # test if reference
        # ref is of form /entrySet/entry/experimentList, which is xpath, using actual tags.
        rtgt  = None
        if "reference" in ctempl and ctempl["reference"] is not None:
             rtgt = ctempl["reference"]
        else:
            #default
            rtgt = template["*"]["*"]["reference"]

        # find current store type (direct/list/index)
        if "store" in ctempl and ctempl["store"] is not None:
            cstore = ctempl["store"]
        else:
            #default
            cstore = template["*"]["*"]["store"]

        if debug:
            print( "  CKEY  ", ckey )
            print( "  CCMPLX", ccomplex )
            print( "  CSTORE", cstore )
            print( "  CREFTG", rtgt )

        if rtgt is not None:
            # add referenced data
            # elem.text: a reference
            # rtgt     : path to referenced dictionary along current path
            #            within record data structure

            #splitting xpath into each tag
            stgt = rtgt.split('/')
            for i in range(1,len(stgt)):
                #checks if each tag is in the rpath that has been passed in this round of recursion.
                if stgt[i] in rpath[i-1]:
                    ##what is supposed to be here??
                    pass
                else:
                    break

            """rewrite as:
            for i in range(len(stgt) - 1):
                if stgt[i] in rpath[i]:
                    pass
                else:
                    break
            """

            #what is the structure and type of rpath?
            #rpath starts as an empty list.
            #this part makes no sense, what i is this?
            #lastmatch is assigned to
            lastmatch = rpath[i-2][stgt[i-1]]
            if cstore == "list":
                if ckey not in rec:
                    rec[ckey] = []
                #appends the information
                rec[ckey].append( lastmatch[stgt[i]][elem.text] )
            elif cstore == "direct":
                rec[ckey] = lastmatch[stgt[i]][elem.text]

        else:
            # build/add current value
            #just assigns cvalue to the text string if not complex
            #if complex, cvalue is a dictionary with the text string udner as value of "value" key
            cvalue = None
            if ccomplex:
                cvalue = {}
                if elem.text:
                    val = str(elem.text).strip()
                    if len(val) > 0:
                        cvalue["value"] = val
            else:
                cvalue = str( elem.text )

            #if ckey in rec:
            #    print(ckey, rec[ckey])

            if cstore  == "direct":
                #rec is a dictionary
                        ##is rec a nested dictionary or is each ckey on the same level?
                        ##can test this and just skip the reference part?
                #ckey is either the tag of the element (first thing after < in xml), or the name as defined in the josn
                #this assigns the actual text of an element to its name.
                #creating new key value pair in rec, key is ckey (tag or name), value is cvalue (text in element)
                rec[ckey] = cvalue
            elif cstore == "list":
                if ckey not in rec:
                    rec[ckey] = []
                    #if list, adds text to the list that is the value of the key (tag or name)
                rec[ckey].append(cvalue)
            elif cstore == "index":
                if ckey not in rec:
                    rec[ckey] = {}

                if "ikey" in ctempl and ctempl["ikey"] is not None:
                    ikey = ctempl["ikey"]
                else:
                    #default
                    ikey = template["*"]["*"]["ikey"]

                if ikey.startswith("@"):
                    #ex iattr = id
                    iattr= ikey[1:]
                    #getting the attribute from the xml tree
                    ikeyval = elem.get(iattr)

                    #inside red dictionary, value of ckey (when store is index) is another dictionary, containing
                    #a key (value of attribute in xml) whose value is cvalue (jsut elem.text or dictionary with elem.text)
                    """
                    ex:
                    xml: <primaryRef db="psi-mi" dbAc="MI:0488" id="MI:0465" refType="identity" refTypeAc="MI:0356"/>
                    json:     "availability":{"entry":{"store":"index","ikey":"@id","name":"availabilityList"}}, (pretend these correspond)

                    ikey = "@id"
                    iattr = "id"
                    ikeyval = "MI:0465"
                    rec[availabilityList(from name)][MI:0465] = **cvalue (what is cvalue in this case; no text)


                    """
                    if ikeyval is not None:
                        rec[ckey][ikeyval] = cvalue

            #create xrefInd inside xref
            #what is this? both json do not have index as a key in ctempl
            if "index" in ctempl and ctempl["index"] is not None:
                ckeyRefInd = ctempl["index"]["name"]
                rec[ckey][ckeyRefInd] = {}

            # parse elem contents

            # add dom attributes
            #elem.attrib probably returns a dictionary with the attributes in xml and their values.
            for cattr in elem.attrib:
                #cvalue can be assumed to be dictionary here because it must be complex if contains attributes.
                #in addition to "value": value, adds each attribute and its value to cvalue.
                cvalue[cattr] = str( elem.attrib[cattr] )

            cpath = []
            #seems like rpath is supposed to containg a list of every tag: cvalue pair up to the current element
            #hence, structure is in the order of rpath list
            #here, we append the current tag: cvalue pair to the end and pass it in for recursion.
            for p in rpath:
                cpath.append(p)
            #append wrapper for later reconstruction too?
            cpath.append( {tag: cvalue })

            if 'index' in ctempl:
                iname = ctempl["index"]["name"]
                ientry = ctempl["index"]["entry"]

            #this is important for defining the structure of this rpath
            #
            for cchld in elem:
                cchldTag = self.modifyTag(cchld, ver)
                if debug:
                    print("  CHLD", cchld.tag);
                #passes in cvalue for rec; can assume it is a dictionary since only the last child
                #will be non complex and will skip all instances of treating it like a dictionary
                #
                self.genericParse( template, ver, cvalue, cpath, cchld)

                if 'index' in ctempl:
                    if cchldTag in ientry and ientry[cchldTag] is not None:
                        keyList = ientry[cchldTag]["key"]

                        kval = []
                        for k in keyList:
                            kvl = cchld.xpath(k)
                            if kvl:
                                kval.append(kvl[0])
                        dbkey = ':'.join(kval)
                        rec[ckey][ckeyRefInd][dbkey] = cvalue[cchldTag][0] if type(cvalue[cchldTag]) is list else cvalue[cchldTag]


            if debug:
                print( json.dumps(self.root, indent=2) )
        return